In [23]:
import win32com.client
import subprocess
import time
import psutil

In [24]:
with open("credentials.txt", "r") as file:
    lines = file.readlines()
    # Read the first line for the client ID and remove extra whitespace
    client_id = lines[0].strip()
    # Read the Second line for the username and remove extra whitespace
    username = lines[1].strip()
    # Read the Third line line for the password and remove extra whitespace
    password = lines[2].strip()
    # Read the Fourth line for the Name and remove extra whitespace
    name = lines[3].strip()

# print(f"Username: {username}")
# print(f"Password: {password}")


In [25]:
def kill_sap_processes():
    sap_processes = ["saplogon.exe", "saplgpad.exe", "sapshcut.exe"]  # common SAP GUI executables
    for proc in psutil.process_iter(['pid', 'name']):
        try:
            if proc.info['name'] and proc.info['name'].lower() in sap_processes:
                print(f"Terminating {proc.info['name']} (PID {proc.info['pid']})")
                proc.terminate()
                try:
                    proc.wait(timeout=5)
                except psutil.TimeoutExpired:
                    print(f"Force killing {proc.info['name']} (PID {proc.info['pid']})")
                    proc.kill()
        except (psutil.NoSuchProcess, psutil.AccessDenied):
            pass

    # short wait before re-launching SAP
    time.sleep(2)


In [26]:
kill_sap_processes()
# session = connect_sap()   # your SAP connection logic

Terminating saplogon.exe (PID 19752)


In [27]:
# open SAP Logon

subprocess.Popen(r"C:\\Program Files (x86)\SAP\\FrontEnd\SAPgui\saplogon.exe")
time.sleep(5)
SapGuiAuto = win32com.client.GetObject('SAPGUI')
application = SapGuiAuto.GetScriptingEngine
connection = application.OpenConnection(name,True)

# login
session = connection.Children(0)
session.findbyId("wnd[0]").maximize
session.findById("wnd[0]/usr/txtRSYST-MANDT").text = client_id # Client
session.findById("wnd[0]/usr/txtRSYST-BNAME").text = username    # User
session.findById("wnd[0]/usr/pwdRSYST-BCODE").text = password  # Password
session.findById("wnd[0]/usr/txtRSYST-LANGU").text = "EN"   # Language
session.findById("wnd[0]").sendVKey(0) # Enter
time.sleep(1)

In [28]:
# navigate to transaction code
session.findById("wnd[0]/tbar[0]/okcd").text = "/nMB52" # Transaction code
session.findById("wnd[0]").sendVKey(0) # Enter
time.sleep(1)

In [29]:
# Entering Plant Number
session.findById("wnd[0]/usr/ctxtWERKS-LOW").text = "9100"
time.sleep(1)
session.findById("wnd[0]/usr/ctxtP_VARI").text = "/US9100-INV."
time.sleep(1)
session.findById("wnd[0]").sendVKey(8) # F8
time.sleep(1)


In [30]:
#### This function presses on the menu bar and triggers the export to spreadsheet option

# --- make sure the ALV grid has focus (menu items are enabled only then)
for pid in [
    "wnd[0]/usr/cntlGRID1/shellcont/shell",
    "wnd[0]/usr/cntlALV_GRID/shellcont/shell",
    "wnd[0]/usr/cntlRESULT_LIST/shellcont/shell",
]:
    try:
        session.findById(pid).setFocus()
        break
    except:
        pass

# --- helper that clicks a menu path by visible captions
def click_menu(*labels):
    path = "wnd[0]/mbar"
    for depth, want in enumerate(labels):
        want = want.lower()
        i = 0
        while True:
            try:
                itm = session.findById(f"{path}/menu[{i}]")
            except:
                raise Exception(f"Menu '{labels[depth]}' not found at depth {depth}.")
            # remove accelerators (&) and match by prefix
            txt = itm.text.replace("&", "").strip().lower()
            if txt.startswith(want):
                path = f"{path}/menu[{i}]"
                break
            i += 1
    # final select
    session.findById(path).select()

# --- List → Export → Spreadsheet…
click_menu("List", "Export", "Spreadsheet")



In [31]:
# --- Export dialog (OK) and Save As
try:
    session.findById("wnd[1]/tbar[0]/btn[0]").press()  # OK on 'Export' popup (if shown)
except: 
    pass

try:
    session.findById("wnd[1]/usr/ctxtDY_PATH").text = r"C:\Temp"
    session.findById("wnd[1]/usr/ctxtDY_FILENAME").text = "MB52_Export.xlsx"
    session.findById("wnd[1]/tbar[0]/btn[0]").press()  # Save
except:
    try:
        session.findById("wnd[2]/tbar[0]/btn[0]").press()  # Overwrite = Yes
    except:
        pass

In [32]:
# Adding Time Stamp as file name
from datetime import datetime
# Function to create time Stamp and add as Text
now = datetime.now()
timestamp = now.strftime("%Y%m%d_%H%M%S")
FileName = "DailyStockCount"
concat = FileName+"_"+timestamp

session.findById("wnd[1]/usr/ssubSUB_CONFIGURATION:SAPLSALV_GUI_CUL_EXPORT_AS:0512/txtGS_EXPORT-FILE_NAME").text = concat

# Clicking Export Button
session.findById("wnd[0]").sendVKey(0) # Enter



In [33]:
session.findById("wnd[1]/usr/ctxtDY_PATH").text =r"C:\\Users\\JeSai\\OneDrive - Boviet Solar USA\Boviet IT Team - Boviet IT Team Channel\BI Dashboards\Stock Data"

# session.findById("wnd[1]").sendVKey(0) # Enter

# Send Tab Key to allow downloading files
# session.findById("wnd[2]").sendVKey(9) #Tab key
# session.findById("wnd[2]").sendVKey(9)
# session.findById("wnd[2]").sendVKey(9)
# session.findById("wnd[2]").sendVKey(9)



In [34]:
session.findById("wnd[1]").sendVKey(0) # Enter


In [35]:
# Allow on the popup window
# session.findById("wnd[2]/usr/btnSPOP-OPTION1").press()


In [36]:
# session.findById("wnd[1]/usr/chkSBOX-MSG_REMEMBER_DECISION").selected = True
# session.findById("wnd[1]/usr/btnSPOP-OPTION1").press()


In [37]:
# def press_allow(session, remember=True):
#     try:
#         wnd = session.findById("wnd[2]")  # security popup is usually wnd[1]
#         usr = wnd.findById("usr")
#         # optionally tick "Remember My Decision"
#         if remember:
#             for c in usr.children:
#                 if c.Type == "GuiCheckBox" and "Remember" in getattr(c, "Text", ""):
#                     c.selected = True
#                     break
#         # press the Allow button
#         for c in usr.children:
#             if c.Type == "GuiButton" and c.Text.strip().lower() == "allow":
#                 c.press()
#                 return True
#         # fallback to common id
#         usr.findById("btnSPOP-OPTION1").press()
#         return True
#     except:
#         return False

# press_allow(session)


In [38]:
# session.findById("wnd[2]/tbar[0]/btn[0]").press()

In [39]:
# session.findById("wnd[0]").sendVKey(0) # Enter


In [ ]:
# Terminate SAP
kill_sap_processes()

Terminating saplogon.exe (PID 19752)
